# Querying and fetching the single-cell data and cell/gene metadata.

This tutorial showcases the easiest ways to query the expression data and cell/gene metadata from the Census, and load them into common in-memory Python objects, including `pandas.DataFrame` and `anndata.AnnData`.

**Contents**

1. Opening the census.
2. Querying expression data.
3. Querying cell metadata (obs).
4. Querying gene metadata (var).

⚠️ Note that the Census RNA data includes duplicate cells present across multiple datasets. Duplicate cells can be filtered in or out using the cell metadata variable `is_primary_data` which is described in the [Census schema](https://github.com/chanzuckerberg/cellxgene-census/blob/main/docs/cellxgene_census_schema.md#repeated-data).

## Opening the census

The `cellxgene_census` python package contains a convenient API to open the latest version of the Census.

In [1]:
import cellxgene_census

In [2]:
census = cellxgene_census.open_soma()

The "stable" release is currently 2024-07-01. Specify 'census_version="2024-07-01"' in future calls to open_soma() to ensure data consistency.


You can learn more about the `cellxgene_census` methods by accessing their corresponding documentation via `help()`. For example `help(cellxgene_census.open_soma)`.

## Querying expression data

A convenient way to query and fetch expression data is to use the `get_anndata` method of the `cellxgene_census` API. This is a method that combines the column selection and value filtering we described above to obtain slices of the expression data based on metadata queries.

The method will return an `anndata.AnnData` object, it takes as an input a census object, the string for an organism, and for both cell and gene metadata we can specify filters and column selection as described above but with the following arguments:

- `obs_column_names` and `var_column_names` — a pair of arguments whose values are lists of strings indicating the columns to select for cell (`obs`) and gene (`var`) metadata respectively.
- `obs_value_filter` —  python expression with selection conditions to fetch **cells** meeting a criteria. For full details see [tiledb.QueryCondition](https://tiledb-inc-tiledb.readthedocs-hosted.com/projects/tiledb-py/en/stable/python-api.html#query-condition).
- `var_value_filter` —  python expression with selection conditions to fetch **genes** meeting a criteria. Details as above.  For full details see [tiledb.QueryCondition](https://tiledb-inc-tiledb.readthedocs-hosted.com/projects/tiledb-py/en/stable/python-api.html#query-condition).


For example if we want to fetch the expression data for:

- Genes `"ENSG00000161798"` and `"ENSG00000188229"`.
- All `"B cells"` of `"lung"` with `"COVID-19"` from non-duplicated cells.
- With all gene metadata and adding `sex` cell metadata.

In [3]:
adata = cellxgene_census.get_anndata(
    census=census,
    organism="Homo sapiens",
    var_value_filter="feature_id in ['ENSG00000161798', 'ENSG00000188229']",
    obs_value_filter="cell_type == 'B cell' and tissue_general == 'lung' and disease == 'COVID-19' and is_primary_data == True",
    obs_column_names=["sex"],
)

And now we can take a look at the results.

In [4]:
adata

AnnData object with n_obs × n_vars = 2729 × 2
    obs: 'sex', 'cell_type', 'tissue_general', 'disease', 'is_primary_data'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_length', 'nnz', 'n_measured_obs'

In [5]:
adata.obs.head()

,sex,cell_type,tissue_general,disease,is_primary_data
0,male,B cell,lung,COVID-19,True
1,male,B cell,lung,COVID-19,True
2,male,B cell,lung,COVID-19,True
3,male,B cell,lung,COVID-19,True
4,male,B cell,lung,COVID-19,True


In [6]:
adata.var.head()

,soma_joinid,feature_id,feature_name,feature_length,nnz,n_measured_obs
0,10507,ENSG00000161798,AQP5,1884,1226640,68915280
1,16091,ENSG00000188229,TUBB4B,2037,26463689,73806975


For a full description of `get_anndata()` refer to `help(cellxgene_census.get_anndata)`

Don't forget to close the census!

## Querying cell metadata (obs)

The human gene metadata of the Census, for RNA assays, is located at `census["census_data"]["homo_sapiens"].obs`. This is a `SOMADataFrame` and as such it can be materialized as a `pandas.DataFrame` via the methods `read().concat().to_pandas()`. See also, the helper function `cellxgene_census.get_obs` which removes some boiler plate.

The mouse cell metadata is at `census["census_data"]["mus_musculus"].obs`.

For slicing the cell metadata there are two relevant arguments that can be passed through `read()`:

- `column_names` — list of strings indicating what metadata columns to fetch. 
- `value_filter` — Python expression with selection conditions to fetch rows, it is similar to [pandas.DataFrame.query()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html), for full details see [tiledb.QueryCondition](https://tiledb-inc-tiledb.readthedocs-hosted.com/projects/tiledb-py/en/stable/python-api.html#query-condition) shortly:
   - Expressions are one or more comparisons
   - Comparisons are one of `<column> <op> <value>` or `<column> <op> <column>`
   - Expressions can combine comparisons using and, or, & or |
   - op is one of < | > | <= | >= | == | != or in

To learn what metadata columns are available for fetching and filtering we can directly look at the keys of the cell metadata.

In [7]:
keys = list(census["census_data"]["homo_sapiens"].obs.keys())

In [8]:
keys

['soma_joinid',
 'dataset_id',
 'assay',
 'assay_ontology_term_id',
 'cell_type',
 'cell_type_ontology_term_id',
 'development_stage',
 'development_stage_ontology_term_id',
 'disease',
 'disease_ontology_term_id',
 'donor_id',
 'is_primary_data',
 'observation_joinid',
 'self_reported_ethnicity',
 'self_reported_ethnicity_ontology_term_id',
 'sex',
 'sex_ontology_term_id',
 'suspension_type',
 'tissue',
 'tissue_ontology_term_id',
 'tissue_type',
 'tissue_general',
 'tissue_general_ontology_term_id',
 'raw_sum',
 'nnz',
 'raw_mean_nnz',
 'raw_variance_nnz',
 'n_measured_vars']

`soma_joinid` is a special `SOMADataFrame` column that is used for join operations. The definition for all other columns can be found at the [Census schema](https://github.com/chanzuckerberg/cellxgene-census/blob/main/docs/cell_census_schema.md#cell-metadata--census_objcensus_dataorganismobs--somadataframe).

All of these can be used to fetch specific columns or specific rows matching a condition. For the latter we need to know the values we are looking for _a priori_.

For example let's see what are the possible values available for `sex`. To this we can load all cell metadata but fetching only for the column `sex`. 

In [10]:
sex_cell_metadata = cellxgene_census.get_obs(census, "homo_sapiens", column_names=["sex"])

In [12]:
sex_cell_metadata.drop_duplicates().reset_index(drop=True)

,sex
0,female
1,male
2,unknown


As you can see there are only three different values for `sex`, that is `"male"`, `"female"` and `"unknown"`. 

With this information we can fetch all cell metatadata for a specific `sex` value, for example `"unknown"`.

In [13]:
cell_metadata_all_unknown_sex = cellxgene_census.get_obs(
    census, "homo_sapiens",
    value_filter="sex == 'unknown'"
)

In [14]:
cell_metadata_all_unknown_sex

,soma_joinid,dataset_id,assay,assay_ontology_term_id,cell_type,cell_type_ontology_term_id,development_stage,development_stage_ontology_term_id,disease,disease_ontology_term_id,...,tissue,tissue_ontology_term_id,tissue_type,tissue_general,tissue_general_ontology_term_id,raw_sum,nnz,raw_mean_nnz,raw_variance_nnz,n_measured_vars
0,63809,94423ec1-21f8-40e8-b5c9-c3ea82350ca4,10x 3' v2,EFO:0009899,dendritic cell,CL:0000451,unknown,unknown,normal,PATO:0000461,...,body of stomach,UBERON:0001161,tissue,stomach,UBERON:0000945,695.0,368,1.888587,12.142867,19550
1,63825,94423ec1-21f8-40e8-b5c9-c3ea82350ca4,10x 3' v2,EFO:0009899,monocyte,CL:0000576,unknown,unknown,normal,PATO:0000461,...,body of stomach,UBERON:0001161,tissue,stomach,UBERON:0000945,6095.0,1427,4.271198,124.798069,19550
2,63829,94423ec1-21f8-40e8-b5c9-c3ea82350ca4,10x 3' v2,EFO:0009899,monocyte,CL:0000576,unknown,unknown,normal,PATO:0000461,...,body of stomach,UBERON:0001161,tissue,stomach,UBERON:0000945,1045.0,492,2.123984,23.318609,19550
3,63842,94423ec1-21f8-40e8-b5c9-c3ea82350ca4,10x 3' v2,EFO:0009899,mast cell,CL:0000097,unknown,unknown,normal,PATO:0000461,...,body of stomach,UBERON:0001161,tissue,stomach,UBERON:0000945,1546.0,640,2.415625,27.823856,19550
4,63845,94423ec1-21f8-40e8-b5c9-c3ea82350ca4,10x 3' v2,EFO:0009899,monocyte,CL:0000576,unknown,unknown,normal,PATO:0000461,...,body of stomach,UBERON:0001161,tissue,stomach,UBERON:0000945,1308.0,530,2.467925,59.814659,19550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3756275,69305276,9f222629-9e39-47d0-b83f-e08d610c7479,Drop-seq,EFO:0008722,ciliated columnar cell of tracheobronchial tree,CL:0002145,unknown,unknown,cystic fibrosis,MONDO:0009061,...,lung,UBERON:0002048,tissue,lung,UBERON:0002048,2748.0,1592,1.726131,15.334125,50205
3756276,69305278,9f222629-9e39-47d0-b83f-e08d610c7479,10x 3' v2,EFO:0009899,alveolar macrophage,CL:0000583,unknown,unknown,interstitial lung disease,MONDO:0015925,...,lung,UBERON:0002048,tissue,lung,UBERON:0002048,6945.0,2010,3.455224,200.698094,50205
3756277,69305280,9f222629-9e39-47d0-b83f-e08d610c7479,10x 3' v3,EFO:0009922,alveolar macrophage,CL:0000583,unknown,unknown,normal,PATO:0000461,...,lung,UBERON:0002048,tissue,lung,UBERON:0002048,37883.0,5559,6.814715,2129.944792,50205
3756278,69305283,9f222629-9e39-47d0-b83f-e08d610c7479,10x 3' v2,EFO:0009899,unknown,unknown,unknown,unknown,normal,PATO:0000461,...,lung,UBERON:0002048,tissue,lung,UBERON:0002048,10531.0,3077,3.422489,227.936529,50205


You can use both `column_names` and `value_filter` to perform specific queries. For example let's fetch the `disease` columns for the `cell_type` `"B cell"` in the `tissue_general` `"lung"` and from non-duplicated cells. 

In [15]:
cell_metadata_b_cell = cellxgene_census.get_obs(
    census,
    "homo_sapiens",
    value_filter="cell_type == 'B cell' and tissue_general == 'lung' and is_primary_data==True",
    column_names=["disease"],
)

In [16]:
cell_metadata_b_cell.value_counts()

disease                                cell_type  tissue_general  is_primary_data
lung adenocarcinoma                    B cell     lung            True               42720
squamous cell lung carcinoma           B cell     lung            True               10631
normal                                 B cell     lung            True                9082
non-small cell lung carcinoma          B cell     lung            True                8742
pulmonary fibrosis                     B cell     lung            True                6798
COVID-19                               B cell     lung            True                2729
chronic obstructive pulmonary disease  B cell     lung            True                2203
lung large cell carcinoma              B cell     lung            True                1534
pulmonary emphysema                    B cell     lung            True                1512
pleomorphic carcinoma                  B cell     lung            True                1210
intersti

## Querying gene metadata (var)

The human gene metadata of the Census is located at `census["census_data"]["homo_sapiens"].ms["RNA"].var`. Similarly to the cell metadata, it is a `SOMADataFrame` and thus we can also use its method `read()`.

The mouse gene metadata is at `census["census_data"]["mus_musculus"].ms["RNA"].var`.

Let's take a look at the metadata available for column selection and row filtering.

In [17]:
keys = list(census["census_data"]["homo_sapiens"].ms["RNA"].var.keys())

In [18]:
keys

['soma_joinid',
 'feature_id',
 'feature_name',
 'feature_length',
 'nnz',
 'n_measured_obs']

With the exception of `soma_joinid` these columns are defined in the [Census schema](https://github.com/chanzuckerberg/cellxgene-census/blob/main/docs/cell_census_schema_0.1.0.md). Similarly to the cell metadata, we can use the same operations to learn and fetch gene metadata.

For example, to get the `feature_name` and `feature_length` of the genes `"ENSG00000161798"` and `"ENSG00000188229"` we can do the following.

In [19]:
gene_metadata = cellxgene_census.get_var(
    census,
    "homo_sapiens",
    value_filter="feature_id in ['ENSG00000161798', 'ENSG00000188229']",
    column_names=["feature_name", "feature_length"],
)

In [20]:
gene_metadata

,feature_name,feature_length,feature_id
0,AQP5,1884,ENSG00000161798
1,TUBB4B,2037,ENSG00000188229


In [21]:
census.close()